# Kunisch Features from ResNet architectures 
## Seminario de Tesis I, Primavera 2022 
### MDS Program. University of Chile.
#### Supervisor: Prof. Benjamín Bustos, Prof. Iván Sipirán
#### Author: Iván Sipirán, modified by Matías Vergara


## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
import time
import os
import copy
import pandas as pd
import math
import random
import shutil

from torch.utils.data import Dataset
from PIL import Image

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np, scipy.io
import argparse
import json


## Mounting Google Drive

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    folder_path = 'drive/MyDrive/TesisMV/'
except:
    folder_path = '../'

In [3]:
os.listdir(folder_path)

['.git',
 '.gitignore',
 '.idea',
 'actas',
 'features',
 'graficos',
 'labels',
 'models',
 'notebooks',
 'patterns',
 'presentaciones',
 'README.md',
 'requirements.txt']

## Dataset and model selection

In [4]:
#modify only this cell
USE_RN50 = False
DS_FLAGS = ['crop', 'elastic', 'rain']
              # 'ref': [invertX, invertY],
              # 'rot': [rotate90, rotate180, rotate270],
              # 'crop': [crop] * CROP_TIMES,
              # 'blur': [blur],
              # 'emboss': [emboss],
              # 'randaug': [randaug],
              # 'rain': [rain],
              # 'elastic': [elastic]
CROP_TIMES = 1
RANDOM_TIMES = 2
ELASTIC_TIMES = 3

In [5]:
# This cells builds the data_flags variable, that will be used
# to map the requested data treatment to folders
MAP_TIMES = {'crop': CROP_TIMES,
         'randaug': RANDOM_TIMES,
         'elastic': ELASTIC_TIMES,
}

DS_FLAGS = sorted(DS_FLAGS)
data_flags = '_'.join(DS_FLAGS) if len(DS_FLAGS) > 0 else 'base'
MULTIPLE_TRANSF = ['crop', 'randaug', 'elastic']
COPY_FLAGS = DS_FLAGS.copy()

for t in MULTIPLE_TRANSF:
    if t in DS_FLAGS:
        COPY_FLAGS.remove(t)
        COPY_FLAGS.append(t + str(MAP_TIMES[t]))
        data_flags = '_'.join(COPY_FLAGS)

patterns_path = folder_path + "patterns/" + data_flags + "/"
labels_path = folder_path + "labels/" + data_flags + "/"
models_path = folder_path + "models/" + (f'resnet50_{data_flags}.pth' if USE_RN50 else f'resnet18_{data_flags}.pth')
features_path = folder_path + "features/" + (f'resnet50_{data_flags}/' if USE_RN50 else f'resnet18_{data_flags}/')

# models_path = folder_path + "models/resnet18_base_each5/resnet18_base_e70.pth"
# features_path = folder_path + "features/resnet18_base_each5/resnet18_base_e70/"

if not (os.path.isdir(patterns_path) and os.path.isdir(labels_path)):
    raise FileNotFoundError("No existen directorios de datos para el conjunto de flags seleccionado. Verifique que el dataset exista y, de lo contrario, llame a Split and Augmentation")

if not (os.path.isfile(models_path)):
    raise FileNotFoundError(f"No se encontró modelo para el conjunto de flags seleccionado. Verifique que el modelo exista y, de lo contrario, llame a ResNet Retraining\n{models_path}")


print("Pattern set encontrado en {}".format(patterns_path))
print("Labels set encontrado en {}".format(labels_path))
print("Modelo encontrado en {}".format(models_path))

Pattern set encontrado en ../patterns/rain_crop1_elastic3/
Labels set encontrado en ../labels/rain_crop1_elastic3/
Modelo encontrado en ../models/resnet18_rain_crop1_elastic3.pth


## Dataset loader

In [6]:
class PatternDataset(Dataset):
    def __init__(self, root_dir, transform=None, build_classification=False, name_cla='output.cla'):
        self.root_dir = root_dir
        self.transform = transform
        self.namefiles = []


        self.classes = sorted(os.listdir(self.root_dir))

        for cl in self.classes:
            for pat in os.listdir(os.path.join(self.root_dir, cl)):
                self.namefiles.append((pat, cl))

        print(f'Files:{len(self.namefiles)}')
        self.namefiles = sorted(self.namefiles, key = lambda x: x[0])

        if build_classification:
            dictClasses = dict()

            for cl in self.classes:
                dictClasses[cl] = []

            for index, (name, cl) in enumerate(self.namefiles):
                dictClasses[cl].append((name, index))

            with open(name_cla, 'w') as f:
                f.write('PSB 1\n')
                f.write(f'{len(self.classes)} {len(self.namefiles)}\n')
                f.write('\n')
                for cl in self.classes:
                    f.write(f'{cl} 0 {len(dictClasses[cl])}\n')
                    for item in dictClasses[cl]:
                        f.write(f'{item[1]}\n')
                    f.write('\n')

    def __len__(self):
        return len(self.namefiles)

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()

        img_name = os.path.join(self.root_dir, self.namefiles[index][1], self.namefiles[index][0])
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return self.namefiles[index], image


## Preparation

In [7]:
random.seed(30)

output_train = features_path + "augmented_train_df.json"
output_val = features_path + "val_df.json"
output_test = features_path + "test_df.json"

train_df = pd.read_json(labels_path + "augmented_train_df.json", orient='index')
val_df = pd.read_json(labels_path + "val_df.json", orient='index')
test_df = pd.read_json(labels_path + "test_df.json", orient='index')

train_pts = train_df.index.values
val_pts = val_df.index.values
test_pts = test_df.index.values

## Extraction

In [8]:
def imshow(inp, title = None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)

    plt.imshow(inp)
    plt.show()

def get_vector(model,layer, dim_embedding, x):

  my_embedding = torch.zeros(dim_embedding)

  def copy_data(m,i,o):
    my_embedding.copy_(o.data.squeeze())

  h = layer.register_forward_hook(copy_data)
  model(x)
  h.remove()

  return my_embedding

device = ('cuda' if torch.cuda.is_available() else 'cpu')

my_transform = transforms.Compose([ transforms.Resize(224),
                                    #transforms.CenterCrop(224),
                     transforms.ToTensor(),
                     transforms.Normalize(mean=[0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])
                    ])

dataTrain = PatternDataset(root_dir=os.path.join(patterns_path, 'train'), transform=my_transform)
dataVal = PatternDataset(root_dir=os.path.join(patterns_path, 'val'), transform=my_transform)
dataTest = PatternDataset(root_dir=os.path.join(patterns_path, 'test'), transform=my_transform)

loaderTrain = DataLoader(dataTrain)
loaderVal = DataLoader(dataVal)
loaderTest = DataLoader(dataTest)

if USE_RN50:
    model = models.resnet50(pretrained = True)
else:
    model = models.resnet18(pretrained = True)
dim = model.fc.in_features
model.fc = nn.Linear(dim, 6)

model = model.to(device)

try:
    model.load_state_dict(torch.load(models_path))
except RuntimeError as e:
    print('Ignoring "' + str(e) + '"')

layer = model._modules.get('avgpool')

model.eval()

#features = []
features_train = {}
features_val = {}
features_test = {}


for name, img in loaderTrain:
  feat = get_vector(model, layer, dim, img.to(device))
  namefile = name[0][0]
  code, rest = namefile.split('.')
  features_train[code] = feat.numpy().tolist()
  #features.append(feat.numpy())

for name, img in loaderVal:
  feat = get_vector(model, layer, dim, img.to(device))
  namefile = name[0][0]
  code, rest = namefile.split('.')
  features_val[code] = feat.numpy().tolist()
  #features.append(feat.numpy())

for name, img in loaderTest:
  feat = get_vector(model, layer, dim, img.to(device))
  namefile = name[0][0]
  code, rest = namefile.split('.')
  features_test[code] = feat.numpy().tolist()
#features = np.vstack(features)
#print(features.shape)

Files:3720
Files:78
Files:78



KeyboardInterrupt



In [ ]:
os.makedirs(features_path, exist_ok=True)

In [ ]:
features_train_df = pd.DataFrame.from_dict(features_train, orient='index')
features_val_df = pd.DataFrame.from_dict(features_val, orient='index')
features_test_df = pd.DataFrame.from_dict(features_test, orient='index')

features_train_df.to_json(output_train, orient='index')
features_val_df.to_json(output_val, orient='index')
features_test_df.to_json(output_test, orient='index')